In [1]:
'''! pip install lightning
! pip install torchmetrics
! pip install watermark
! pip install mlxtend
! pip install tensorboard
! pip install pandas'''

'! pip install lightning\n! pip install torchmetrics\n! pip install watermark\n! pip install mlxtend\n! pip install tensorboard\n! pip install pandas'

In [1]:
import time

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchvision import datasets, transforms

import lightning as L
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import TQDMProgressBar
from lightning.pytorch.callbacks import ModelCheckpoint
import torchmetrics

import timm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from datamodules import RSNAdataset
from plotting import show_failures, plot_loss_and_acc

/home/karanjotvendal/miniforge3/envs/thesis/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import wandb
wandb.login(key='a2a7828ed68b3cba08f2703971162138c680b664')

In [2]:
%reload_ext watermark
%watermark -a 'Karanjot Vendal' -v -p torch --iversion

Author: Karanjot Vendal

Python implementation: CPython
Python version       : 3.11.4
IPython version      : 8.14.0

torch: 2.0.1

lightning   : 2.0.6
timm        : 0.9.2
pandas      : 2.0.3
numpy       : 1.25.1
matplotlib  : 3.7.2
torchmetrics: 1.0.1
torchvision : 0.15.2
torch       : 2.0.1



# Hyperparameters

In [2]:
BATCH_SIZE = 10
NUM_EPOCHS = 10
LEARNING_RATE = 0.0001

#DATASET = 'MNIST' #CIFAR or MNIST
NUM_WORKERS = 6
GRAYSCALE = True
NUM_CLASSES = 2

In [ ]:
run = wandb.init(
      # Set the project where this run will be logged
      project="Kaggle LB-2 Baseline run", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"experiment_{2}", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": LEARNING RATE,
      "architecture": "CNN-LSTM",
      "dataset": "MICAA MRI",
      "epochs": NUM_EPOCHS,
      "BATCH_SIZE": BATCH_SIZE
      })

# Model

In [3]:
class RecNet(nn.Module):
    def __init_(self):
        super().__init__()
        
        #self.CNN = timm.create_model('resnet50', pretrained=True, num_classes=0)
        self.CNN = timm.create_model('resnet50', pretrained=True, num_classes=0, in_chans=1)
        in_feature = self.CNN.fc.in_feature
        
        self.rnn = nn.GRU(input_size=in_feature, hidden_size=64, batch_first= True, bidirectional=False)
        
        self.fc = self.Linear(hidden_size, 32, bias=True)
        self.classifier = self.Linear(32, num_calsses=2, bias=True)

    def forward(self, x, mask_in, mask_dup):
        mask = mask_layer(mask_in, mask_dup)
        
        out = self.CNN(x)
        out = self.RNN(out)
        out = out * mask
        out = self.fc(out)

        logits = self.classifier(out)
        output = F.softmax(logits, dim=1)
        #output = F.softmax(logits) #[prob 0, prob 1]

    def mask_layer(self, mask_in, mask_dup):
        mask_1 = torch.ones(mask_in, 64)
        mask_0 = torch.zeros(mask_dup, 64)
        
        return torch.cat((mask_1, mask_0), 0)

# configuring the lightning module

In [4]:
#Configuring the lightning module
class LightningModel(L.LightningModule):
    def __init__(self, model, learning_rate, num_epochs, batch_size, dataset, grayscale, height_width):
        super().__init__()

        self.learning_rate = learning_rate
        self.model = model
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.dataset = dataset
        self.height_width = height_width

        self.save_hyperparameters(ignore=["model"])

        self.train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=10)
        self.val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=10)
        self.test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=10)
        
        self.train_f1 = torchmetrics.F1Score(task="multiclass", num_classes=2)
        self.val_f1 = torchmetrics.F1Score(task="multiclass", num_classes=2)
        self.test_f1 = torchmetrics.F1Score(task="multiclass", num_classes=2)

        self.train_auroc = AUROC(task="multiclass", num_classes=2)
        self.val_auroc = AUROC(task="multiclass", num_classes=2)
        self.test_auroc = AUROC(task="multiclass", num_classes=2)

    def forward(self, x):
        return self.model(x, mask_in, mask_dup)

    def _shared_step(self, batch):
        features, true_labels = batch
        out = self(features)

        loss = F.cross_entropy(out, true_labels)
        predicted_labels = torch.argmax(out, dim=1)
        return loss, true_labels, predicted_labels

    def training_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)

        self.log("train_loss", loss)
        self.train_acc(predicted_labels, true_labels)
        self.log(
            "train_acc", self.train_acc, prog_bar=True, on_epoch=True, on_step=False
        )
        return loss

    def validation_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)

        self.log("val_loss", loss, prog_bar=False)
        self.val_acc(predicted_labels, true_labels)
        self.log("val_acc", self.val_acc, prog_bar=False)

    def test_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)
        self.test_acc(predicted_labels, true_labels)
        self.log("test_acc", self.test_acc)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

# Datamodule

In [ ]:
class MriDataModule(L.LightningDataModule):
    def __init__(self, data_path="./data/reduced_dataset/", batch_size=10, num_workers=0, height_width=(112,112), mod="FLAIR"):
        super().__init__()
        self.batch_size = batch_size
        self.data_path = data_path
        self.height_width = height_width
        self.num_workers = num_workers

    def prepare_data(self):
        
        self.train_transform = transforms.Compose(
            [
                transforms.Resize(self.height_width),
                transforms.ToTensor(),
                transforms.Normalize((0.5), (0.5))
            ]
        )

        self.test_transform = transforms.Compose(
            [
                transforms.Resize(self.height_width),
                transforms.ToTensor(),
                transforms.Normalize((0.5), (0.5))
            ]
        )
        return

    def setup(self, stage=None):
        # Note transforms.ToTensor() scales input images
        # to 0-1 range
        folds_xtrain = np.load('./data/folds/xtrain.npy', allow_pickle=True)
        folds_xtest = np.load('./data/folds/xtest.npy', allow_pickle=True)
        folds_ytrain = np.load('./data/folds/ytrain.npy', allow_pickle=True)
        folds_ytest = np.load('./data/folds/ytest.npy', allow_pickle=True)
        
        xtrain = folds_xtrain[0]
        ytrain = folds_ytrain[0]
        xtest = folds_xtest[0]
        ytest = folds_ytest[0]

        
        self.train = RSNAdataset(
            'data/reduced_dataset/',
            xtrain,  
            ytrain,
            n_slices=254,
            img_size=112,
            transform=None
        )

        '''self.test = datasets(
            root=self.data_path,
            train=False,
            transform=self.test_transform,
            download=False,
        )'''

        #self.train, self.valid = random_split(train, lengths=[55000, 5000])

    def train_dataloader(self):
        train_loader = DataLoader(
            dataset=self.train,
            batch_size=self.batch_size,
            drop_last=False,
            shuffle=False,
            num_workers=self.num_workers,
        )
        return train_loader

    '''def val_dataloader(self):
        valid_loader = DataLoader(
            dataset=self.valid,
            batch_size=self.batch_size,
            drop_last=False,
            shuffle=False,
            num_workers=self.num_workers,
        )
        return valid_loader

    def test_dataloader(self):
        test_loader = DataLoader(
            dataset=self.test,
            batch_size=self.batch_size,
            drop_last=False,
            shuffle=False,
            num_workers=self.num_workers,
        )
        return test_loader'''

# preparing the dataset

In [6]:
dm = MriDataModule(batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
dm.prepare_data()
dm.setup()

# Inspecting the dataset

In [ ]:
from collections import Counter

train_counter = Counter()
for images, labels in dm.train_dataloader():
    train_counter.update(labels.tolist())

#test_counter = Counter()
#for images, labels in dm.test_dataloader():
#    test_counter.update(labels.tolist())

print("\nTraining label distribution:")
sorted(train_counter.items())

#print("\nTest label distribution:")
#sorted(test_counter.items())

In [ ]:
for images, labels in dm.train_dataloader():  
    break
'''
plt.figure(figsize=(8, 8))
plt.axis("off")
plt.title("Training images")
plt.imshow(np.transpose(torchvision.utils.make_grid(
    images[:64], 
    padding=2,
    normalize=True),
    (1, 2, 0)))
plt.show()'''
print(images[0].shape)

# setting up trainer and logger

In [ ]:
#config lighnting module
pymodel = resnet18(num_classes=NUM_CLASSES)
lightning_model = LightningModel(pymodel, learning_rate=LEARNING_RATE, batch_size=BATCH_SIZE, num_epochs=NUM_EPOCHS, dataset=DATASET, grayscale=GRAYSCALE,
                                 height_width=height_width)

#checkpointing the best model
#configuring earlystopping
callbacks = [ModelCheckpoint(save_top_k=1, mode='max', monitor='val_acc'),
            TQDMProgressBar(refresh_rate=50),
            EarlyStopping(monitor="val_loss", mode="min", patience=10)
            ]

#configuring logger
csv_logger = CSVLogger(save_dir='csv_logs/', name='ResNet18')
tb_logger = TensorBoardLogger(save_dir='tb_logs/', name='ResNet18')

# Training

In [ ]:
trainer = L.Trainer(
    max_epochs=NUM_EPOCHS,
    callbacks = callbacks,
    accelerator="gpu",
    devices=1,
    logger=[csv_logger, tb_logger],
)

In [ ]:
start_time = time.time()

trainer.fit(model=lightning_model, datamodule=dm)
runtime = (time.time() - start_time)/60
print(f'Training finished in {runtime: .2f} min in total')

# Evaluating the model